## 1. youtube list 추출

In [1]:
!pip install google-api-python-client

In [46]:
#사용할library

import os
import datetime as dt
from apiclient.discovery import build
from apiclient.errors import HttpError
import csv
import pandas
from tqdm import tqdm

In [47]:
#전역변수 
# api_key = os.environ.get("APIKEY_GOOGLE")    
api_obj = build('youtube','v3',developerKey='MY_API_KEY')

In [48]:
#youtube_list만들기
def youtube_list(query,start_date, end_date, order_option='relevance'):

    res_list =[]
    cnt = 0

    try:
        response = api_obj.search().list(part='id,snippet',q=query,publishedAfter=start_date, publishedBefore=end_date,order=order_option, maxResults = 100).execute()
        now = dt.datetime.now()
        nowDatetime = now.strftime('%Y%m%d(%H_%M_%S)')
        print(nowDatetime)
    except HttpError as e:
        print(e)
        return 
        

    while response:
        for i,item in tqdm(enumerate(response['items'])):
            #video만 playlist channel제외
            if item['id']['kind']=='youtube#video':
                res_list.append([i, item['id']['videoId'], 'https://www.youtube.com/watch?v={id}'.format(id=item['id']['videoId']), item['snippet']['publishedAt'], item['snippet']['title']])
                
        #다음페이지가 있으면 api호출을 한다
        if 'nextPageToken' in response:
            cnt += 1
            print("nextpage:",cnt)
            try:
                response = api_obj.search().list(part='id,snippet',q=query,publishedAfter=start_date, publishedBefore=end_date,order=order_option, pageToken=response['nextPageToken'], maxResults=100).execute()
                
            except  HttpError as e:
                print(e)
                pass
        else:
            break

    
    return res_list,nowDatetime


In [49]:
######################################list파일작성 메인함수
query = "저출산"
start_date = '2022-06-01T00:00:00Z'  
end_date = '2023-05-31T23:59:59Z' 
order_option = 'viewCount' 

#order option
#date – 리소스를 만든 날짜를 기준으로 최근 항목부터 시간 순서대로 리소스를 정렬합니다.
#rating – 높은 평가부터 낮은 평가순으로 리소스를 정렬합니다.
#relevance – 검색 쿼리에 대한 관련성을 기준으로 리소스를 정렬합니다. 이 매개변수의 기본값입니다.
#title – 제목에 따라 문자순으로 리소스를 정렬합니다.
#videoCount – 업로드한 동영상 수에 따라 채널을 내림차순으로 정렬합니다.
#viewCount – 리소스를 조회수가 높은 항목부터 정렬합니다.


y_list, searchdatetime= youtube_list(query,start_date, end_date, order_option)
df = pandas.DataFrame(y_list)
filename = './data/0_{}_{}_{}_{}.csv'.format(query,start_date[:7],end_date[:7],searchdatetime)  #조회일시를 파일명으로 기록합니다
df.to_csv(filename, encoding="utf-8-sig")

20230604(22_24_30)


50it [00:00, 112629.00it/s]

nextpage: 1



50it [00:00, 120525.98it/s]


nextpage: 2


...

50it [00:00, 158275.62it/s]


nextpage: 9


50it [00:00, 202427.80it/s]


nextpage: 10


8it [00:00, 40184.95it/s]


In [54]:
######################################list파일작성 메인함수
query = "저출산"
start_date = '2022-06-01T00:00:00Z'  
end_date = '2023-05-31T23:59:59Z' 
order_option = 'viewCount' 

#order option
#date – 리소스를 만든 날짜를 기준으로 최근 항목부터 시간 순서대로 리소스를 정렬합니다.
#rating – 높은 평가부터 낮은 평가순으로 리소스를 정렬합니다.
#relevance – 검색 쿼리에 대한 관련성을 기준으로 리소스를 정렬합니다. 이 매개변수의 기본값입니다.
#title – 제목에 따라 문자순으로 리소스를 정렬합니다.
#videoCount – 업로드한 동영상 수에 따라 채널을 내림차순으로 정렬합니다.
#viewCount – 리소스를 조회수가 높은 항목부터 정렬합니다.


y_list, searchdatetime= youtube_list(query,start_date, end_date, order_option)
df = pandas.DataFrame(y_list)
filename = './data/0_{}_{}_{}_{}.csv'.format(query,start_date[:7],end_date[:7],searchdatetime)  #조회일시를 파일명으로 기록합니다
df.to_csv(filename, encoding="utf-8-sig")

20230604(22_25_47)


50it [00:00, 78486.23it/s]

nextpage: 1



50it [00:00, 131896.35it/s]


nextpage: 2


...

50it [00:00, 198406.05it/s]


nextpage: 9


50it [00:00, 112387.57it/s]


nextpage: 10


7it [00:00, 37641.19it/s]


In [55]:
len(y_list)

507

## 2. youtube 조회수 상위 50개 댓글 크롤링

In [56]:
def make_comments(video_id):

    #댓글 및 대댓글 저장 리스트
    comments = []
    
    #nextPageToken 초기화
    next_page_token = None

    
    # 대댓글 가져오기
    def get_replies(parent_id, next_page_token=None):
        has_more_replies = True
        while has_more_replies:
            replies_response = api_obj.comments().list(
                part='snippet',
                parentId=parent_id,
                maxResults=100,
                pageToken=next_page_token
            ).execute()
        
            for reply_item in replies_response['items']:
                comments.append([reply_item['snippet']['textOriginal'], reply_item['snippet']['authorDisplayName'], reply_item['snippet']['publishedAt'], reply_item['snippet']['likeCount'], 'Reply'])
        
            if 'nextPageToken' in replies_response:
                next_page_token = replies_response['nextPageToken']
            else:
                has_more_replies = False
    

    # 댓글 및 대댓글 가져오기
    has_more_comments = True
    while has_more_comments:
        try:  #수정위치
            response = api_obj.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=100,
                pageToken=next_page_token
            ).execute()
  
        except HttpError as e:
            print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))
            has_more_comments = False
            break
   
    
        for item in response['items']:
            # 댓글 정보 저장
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([comment['textOriginal'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount'], 'Comment'])
    
            # 대댓글 정보 저장
            if 'totalReplyCount' in item['snippet']:
                total_reply_count = item['snippet']['totalReplyCount']
                if total_reply_count > 0:
                    get_replies(item['snippet']['topLevelComment']['id'], next_page_token)
    
        if 'nextPageToken' in response:
            next_page_token = response['nextPageToken']
        else:
            has_more_comments = False

    # 이스케이프 문자 설정 예시
    escapechar = '\\'  # 이스케이프 문자로 역슬래시를 사용
            
    
    #댓글 저장하기 
    with open('./data/comments_{}.csv'.format(video_id), 'w', encoding='utf-8-sig', newline='') as file:
        writer = csv.writer(file, escapechar=escapechar)  # writer 객체에 이스케이프 문자 설정
        writer.writerow(['Comment', 'Author', 'Published At', 'Likes', 'Type'])
        writer.writerows(comments)

    print(f'Total comments: {len(comments)}')

In [57]:
################comment만드는 메인함수
for index_y_id in tqdm(range(len(y_list))):  
    if index_y_id >4:
        break
    make_comments(y_list[index_y_id][1])         

  0%|          | 1/507 [00:59<8:23:25, 59.70s/it]

Total comments: 8241


  0%|          | 2/507 [01:10<4:19:04, 30.78s/it]

Total comments: 911


  1%|          | 3/507 [04:37<15:36:51, 111.53s/it]

Total comments: 18920


  1%|          | 4/507 [05:12<11:20:10, 81.13s/it] 

Total comments: 3185


  1%|          | 5/507 [05:37<9:25:04, 67.54s/it] 

Total comments: 1860


In [38]:
# 한개씩 처리시 (오류나는 영상 댓글허용안되는)
# make_comments("DTh1mRLTHFM")

An HTTP error 403 occurred:
b'{\n  "error": {\n    "code": 403,\n    "message": "The video identified by the \\u003ccode\\u003e\\u003ca href=\\"/youtube/v3/docs/commentThreads/list#videoId\\"\\u003evideoId\\u003c/a\\u003e\\u003c/code\\u003e parameter has disabled comments.",\n    "errors": [\n      {\n        "message": "The video identified by the \\u003ccode\\u003e\\u003ca href=\\"/youtube/v3/docs/commentThreads/list#videoId\\"\\u003evideoId\\u003c/a\\u003e\\u003c/code\\u003e parameter has disabled comments.",\n        "domain": "youtube.commentThread",\n        "reason": "commentsDisabled",\n        "location": "videoId",\n        "locationType": "parameter"\n      }\n    ]\n  }\n}\n'
Total comments: 0
